# Install Java

In [ ]:
!sudo apt-get update
!sudo apt-get install openjdk-11-jdk -y

In [ ]:
!java -version

In [ ]:
!readlink -f $(which java)

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Install HBase

In [ ]:
!wget https://downloads.apache.org/hbase/2.5.10/hbase-2.5.10-bin.tar.gz
!tar xvf hbase-2.5.10-bin.tar.gz

In [ ]:
! mv hbase-2.5.10 /opt/hbase

In [4]:
!sed -i 's|# export JAVA_HOME=.*|export JAVA_HOME=/usr/lib/jvm/java-11-openjdk-amd64|' /opt/hbase/conf/hbase-env.sh

In [1]:
# Verify JAVA_HOME
import os

# Check if JAVA_HOME is set, if not set it to the correct path
java_home = '/usr/lib/jvm/java-11-openjdk-amd64'
if not os.path.exists(java_home):
    raise FileNotFoundError(f'Java path not found: {java_home}')
os.environ['JAVA_HOME'] = java_home
os.environ['PATH'] = java_home + '/bin:' + os.environ['PATH']


!java -version

openjdk version "11.0.24" 2024-07-16
OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu324.04.1)
OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu324.04.1, mixed mode, sharing)


In [2]:
#Start HBase
!/opt/hbase/bin/start-hbase.sh

running master, logging to /opt/hbase/bin/../logs/hbase-emma-master-emma-Inspiron-3501.out


In [4]:
!jps

385472 Jps
385096 HMaster


# Install Spark

In [ ]:
!wget -v https://dlcdn.apache.org/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

In [ ]:
!tar xvf spark-3.4.3-bin-hadoop3.tgz

In [ ]:
!pip3 install findspark --break-system-packages

In [ ]:
! mv spark-3.4.3-bin-hadoop3 /opt/spark

In [5]:
import os
import findspark

os.environ["SPARK_HOME"] = "/opt/spark"

# Initialize PySpark
findspark.init()


# Collect data

Download csv file from https://obis.org/data/access/#

In [ ]:
!echo "create 'OBIS', 'cf'" | /opt/hbase/bin/hbase shell

In [6]:
from pyspark.sql import SparkSession

# Start Spark Session with HBase connector
spark = SparkSession.builder \
    .appName("HBase-PySpark-Connection") \
    .config("spark.jars.packages", "org.apache.hbase.connectors.spark:hbase-spark:1.0.1") \
    .getOrCreate()

24/09/24 12:02:17 WARN Utils: Your hostname, emma-Inspiron-3501 resolves to a loopback address: 127.0.1.1; using 193.10.38.136 instead (on interface wlp0s20f3)
24/09/24 12:02:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/emma/.ivy2/cache
The jars for the packages stored in: /home/emma/.ivy2/jars
org.apache.hbase.connectors.spark#hbase-spark added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f9cdd4e1-cb3b-4295-9233-109f0610d892;1.0
	confs: [default]
	found org.apache.hbase.connectors.spark#hbase-spark;1.0.1 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found org.apache.hbase.thirdparty#hbase-shaded-miscellaneous;4.1.4 in central
	found com.google.errorprone#error_prone_annotations;2.18.0 in central
	found org.apache.hbase#hbase-shaded-client;2.5.4 in central
	found io.opentelemetry#opentelemetry-semconv;1.15.0-alpha in central
	found io.opentelemetry#opentelemetry-api;1.15.0 in central
	found io.opentelemetry#opentelemetry-context;1.15.0 in central
	found commons-logging#commons-logging;1.2 in central
	found org.apache.htrace#htrace-core4;4.1.0-incubating in central
	found org.apache.yetus#audience-annotations;0.5.0 in centr

In [7]:
spark

In [8]:
df = spark.read \
    .format("org.apache.hbase.connectors.spark") \
    .option("hbase.table.name", "OBIS") \
    .load()

df.show()

Py4JJavaError: An error occurred while calling o35.load.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: org.apache.hbase.connectors.spark. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:738)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:208)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:172)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodA

In [9]:
spark.stop()

In [10]:
#Stop HBase
!/opt/hbase/bin/stop-hbase.sh

stopping hbase............


In [11]:
!jps

387684 Jps
385555 SparkSubmit
